# Example of Database Queries

20210226 : 

In [1]:
# Imports
import os, sys
module_path = os.path.abspath(os.path.join('/Users/Phil/Documents/jupyterlab_sqlalchemy/modules'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
from platform import python_version
# from sqlalchemy import and_, or_

from models_chinook import Album, Artist
from models_pab import Owner, Pet
import db_config as db

basedir = /Users/Phil/Documents/jupyterlab_sqlalchemy


## Setup connections to databases

In [2]:
# Input app directory and database filename
answer = input("Do you want to run production or development'? Enter 'prod' or 'dev': ") 
# answer = "dev"
if answer == "prod": 
    run_config = 'production'
elif answer == "dev":
    run_config = 'development'
else: 
    print("Please enter 'prod' or 'dev'.")
    raise

db.set_engines(db.config[run_config])
db.set_session()

Do you want to run production or development'? Enter 'prod' or 'dev':  dev


## Session and Engines

In [3]:
db.session

In [4]:
db.config[run_config].DB1, db.config[run_config].DB2

('chinook.sqlite', 'pab.sqlite')

In [5]:
db.get_engine(Album)

Engine(sqlite:////Users/Phil/Documents/jupyterlab_sqlalchemy/chinook.sqlite)

In [6]:
db.get_engine(Owner)

Engine(sqlite:////Users/Phil/Documents/jupyterlab_sqlalchemy/pab.sqlite)

## Example Queries

### Database 1 : chinook.sqlite

In [7]:
table = Album
q = db.session.query(table)
pd.read_sql(q.statement, db.get_engine(table))

,id,title,artistid
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
...,...,...,...
342,343,Respighi:Pines of Rome,226
343,344,Schubert: The Late String Quartets & String Qu...,272
344,345,Monteverdi: L'Orfeo,273
345,346,Mozart: Chamber Music,274


In [8]:
table = Artist
q = db.session.query(table)
pd.read_sql(q.statement, db.get_engine(table))

,id,name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


In [9]:
q = db.session.query(Album, Artist.name)\
    .filter(Album.artistid == Artist.id)
pd.read_sql(q.statement, db.get_engine(Artist))

,id,title,artistid,name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,Accept
2,3,Restless and Wild,2,Accept
3,4,Let There Be Rock,1,AC/DC
4,5,Big Ones,3,Aerosmith
...,...,...,...,...
342,343,Respighi:Pines of Rome,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,Nash Ensemble


In [10]:
q = db.session.query(Album, Artist.name)\
    .filter(Album.artistid == Artist.id)\
    .filter(Artist.name == "AC/DC")
pd.read_sql(q.statement, db.get_engine(Artist))

,id,title,artistid,name
0,1,For Those About To Rock We Salute You,1,AC/DC
1,4,Let There Be Rock,1,AC/DC


### Database 2 : pab.sqlite

In [11]:
table = Owner
q = db.session.query(table)
pd.read_sql(q.statement, db.get_engine(table))

,id,name,email
0,1,Michael Mitchell,mm@mail.com
1,2,Robert King,rk@mail.com
2,3,Laura Callahan,lc@mail.com
3,4,Leonie Kohler,lk@mail.com


In [12]:
table = Pet
q = db.session.query(table)
pd.read_sql(q.statement, db.get_engine(table))

,id,name,animal,owner_id,weight_lb,weight_kg,weight_st
0,1,Baron,dog,3,85,38.548753,6.071429
1,2,Auzzie,dog,2,35,15.873016,2.500000


## Example : add an object and set values for computed columns

In [13]:
new_pet = Pet(
    name = "Joe",
    animal = "goldfish",
    owner_id = 2,
    weight_lb = 0.125
)
db.session.add(new_pet)
db.session.commit()

In [14]:
q = db.session.query(table)
pd.read_sql(q.statement, db.get_engine(table))

,id,name,animal,owner_id,weight_lb,weight_kg,weight_st
0,1,Baron,dog,3,85.000,38.548753,6.071429
1,2,Auzzie,dog,2,35.000,15.873016,2.500000
2,3,Joe,goldfish,2,0.125,NaN,NaN


In [15]:
# test setting the value of computed columns which are weight_kg and weight_st for the pet model
new_pet.set_computed_columns()
db.session.commit()

In [16]:
pd.read_sql(q.statement, db.get_engine(table))

,id,name,animal,owner_id,weight_lb,weight_kg,weight_st
0,1,Baron,dog,3,85.000,38.548753,6.071429
1,2,Auzzie,dog,2,35.000,15.873016,2.500000
2,3,Joe,goldfish,2,0.125,0.056689,0.008929


In [17]:
# delete "Joe" so that I can re-run this notebook without accumulating pets
db.session.delete(new_pet)
db.session.commit()
pd.read_sql(q.statement, db.get_engine(table))

,id,name,animal,owner_id,weight_lb,weight_kg,weight_st
0,1,Baron,dog,3,85,38.548753,6.071429
1,2,Auzzie,dog,2,35,15.873016,2.500000


## Close sqlalchemy session

In [18]:
db.session.close()